<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [8]:
import re
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import PorterStemmer,SnowballStemmer,LancasterStemmer

from pymystem3 import Mystem 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression

from catboost import CatBoostClassifier,Pool
from catboost.text_processing import Tokenizer

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix

## Подготовка

In [9]:
df = pd.read_csv('/datasets/toxic_comments.csv',index_col=0,encoding='utf-8')
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [10]:
%%time
mst = Mystem()
lst = LancasterStemmer()
stemmer = lst.stem

def lemmatize(text):
    text = ''.join( re.sub(r"([^a-z\'])+",' ',text.lower()) )
#     print(text)
#     text = re.sub('( )+',' ',text)
    return ''.join( stemmer(text)).strip('\n')                

# lemmatize(df.text[2])                    
df['lemmas'] = df.text.apply(lemmatize)



CPU times: user 4.1 s, sys: 16.4 ms, total: 4.12 s
Wall time: 4.12 s


In [11]:
%%time

tr,te = train_test_split(df, test_size = .25, shuffle = True)
X = 'lemmas'  
y = 'toxic'


CPU times: user 29.5 ms, sys: 10.4 ms, total: 39.9 ms
Wall time: 38.3 ms


## Обучение

In [12]:
text_options =  {
    "tokenizers" : [{
        "tokenizer_id" : "Space",
        "delimiter" : " ,",
        "lowercasing" : "true",
#        "lemmatizing" : "true",
        "split_by_set": "true"                    
    },{
    'tokenizer_id': 'Sense',
    'separator_type': 'BySense'
    }],

    "dictionaries" : [
        {"dictionary_id" : "BiGram",
            "max_dictionary_size" : "100000",
            "occurrence_lower_bound" : "15",
            "gram_order" : "2"
        },{
        "dictionary_id" : "Word",
        "occurrence_lower_bound" : "20",
        "gram_order" : "1"
        },
        {"dictionary_id" : "TetraGram",
            "max_dictionary_size" : "100000",
            "occurrence_lower_bound" : "10",
            "gram_order" : "4"
        }
    ],
    "feature_processing" : {
        "default" : [{
            "dictionaries_names" : ["Word","BiGram",'TetraGram'],
            "feature_calcers" : ["BoW"],
            "tokenizers_names" : ["Sense"]
        }]
        
    }
}
cbm = CatBoostClassifier(learning_rate = .075,n_estimators =8192 ,
                         bootstrap_type='Bernoulli',subsample=.2,
                         text_processing = text_options,
                         verbose= 128, eval_metric='F1',random_state=499)

In [13]:
p_tr = Pool(tr[[X]],tr[y], text_features = [X] ) 
p_te = Pool( te[[X]], text_features = [X] )
cbm.fit(p_tr)    


0:	learn: 0.2703551	total: 352ms	remaining: 47m 59s


KeyboardInterrupt: 

In [ ]:
%%time
cbm.fit(tr[X],tr[y],verbose=128)

In [ ]:
pr = cbm.predict(p_te)
confusion_matrix(te[y],pr)

In [ ]:
for t in np.arange(.2,.6,.025):
    cbm.set_probability_threshold(t)
    pr = cbm.predict(te[X])
    #confusion_matrix(te[y],pr)
    
    s = f"{round(t,2)} -->\t"
    for metric in [f1_score,accuracy_score,precision_score,recall_score]:
        s+=f"{metric.__name__}:{round( metric(te[y],pr), 4)}\t"
    print(s)        

In [ ]:
cbm.set_probability_threshold(.33)
pr = cbm.predict(te[X])
for metric in [f1_score,accuracy_score,precision_score,recall_score]:
    print( f"{metric.__name__}:{round( metric(te[y],pr), 4)}\t" )
"confusion_matrix:\n",confusion_matrix(te[y],pr)


In [ ]:
# %%time
# params_grid ={
#     'learning_rate':[0.3,.1,.03,.01],
#     'auto_class_weights':['Balanced']
# }
# cbc = GridSearchCV( 
#     CatBoostClassifier( eval_metric='F1',text_features = X,tokenizers=verbose= 100  ),
#     params_grid,
#     scoring='f1_weighted',
#     cv= 4)  
# cbc.fit(tr[X],tr[y])

In [ ]:
cbc.cv_results_

In [ ]:
pr = cbc.predict(te[X])

In [ ]:
model =  CatBoostClassifier( eval_metric='F1',eval_period=50,text_features = X,verbose= 100,n_estimators=500 , learning_rate=.03 )

## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны